In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

from functools import partial
from ipywidgets import interact
from IPython.display import display, clear_output
from io import StringIO

from basicsr.utils import tensor2img
import os
from basicsr.data.paired_image_dataset import PairedImageDataset
from ipyfilechooser import FileChooser
from ipywidgets import Layout, GridBox, Box, HBox, VBox, ValueWidget, Label, Text, fixed
import ipywidgets as widgets

import pprint
import numpy as np

def adjusted_display_images(slice, dset):
    train_dict = dset[slice]
    
    lq = tensor2img(train_dict['lq'], rgb2bgr=False)
    gt = tensor2img(train_dict['gt'], rgb2bgr=False)
    
    # Check if the images are grayscale or RGB
    cmap_lq = 'gray' if lq.ndim == 2 else None
    cmap_gt = 'gray' if gt.ndim == 2 else None
    
    # Create a single figure with two subplots
    fig, axes = plt.subplots(1, 2, figsize=(12, 5), num=1, clear=True)
    # Display lq image on the first subplot
    axes[0].imshow(lq, cmap=cmap_lq)
    axes[0].set_title("LQ")
    axes[0].axis("off")

    # Display gt image on the second subplot
    axes[1].imshow(gt, cmap=cmap_gt)
    axes[1].set_title("GT")
    axes[1].axis("off")

    plt.show()

# Event handler to update the opt dictionary when text input changes
def update_opt(change, key):
    try: 
        num = int(change["new"])    
        opt[key] = num
    except:
        opt[key] = change["new"]

def on_folder_change(change, key):
    opt[key] = change.selected


# Define the initial opt dictionary
opt = {}
current_directory = os.getcwd()

opt["dataroot_gt"] = current_directory 
opt["dataroot_lq"] = current_directory 

opt["gt_size"] = 96
opt["scale"] = 1
opt["phase"] = "train"
opt["use_hflip"] = False
opt["use_rot"] = False

widgets_list = []
text_layout = Layout(width="500px")  # adjust the value as needed

dset = None

fc_lq = FileChooser(
    path=current_directory,  # starting directory
    show_only_dirs=True,  # only show directories
    title="LQ folder path",
)

fc_gt = FileChooser(
    path=current_directory,  # starting directory
    show_only_dirs=True,  # only show directories
    title="GT folder path",
)

fc_lq.register_callback(partial(on_folder_change, key='dataroot_lq'))
fc_gt.register_callback(partial(on_folder_change, key='dataroot_gt'))

widgets_list.append(fc_gt)
widgets_list.append(fc_lq)

for idx, (key, value) in enumerate(opt.items()):
    if idx < 2:
        continue
    if key == 'io_backend':
        continue

    txt = widgets.Text(value=str(value), description=key, layout=text_layout)
    txt.observe(lambda change, k=key: update_opt(change, k), names="value")
    widgets_list.append(txt)

out = widgets.Output()  # Your main output widget

def initialize_and_display_slider(dset):
    dset_length = len(dset)-1 if dset else 0
    w = widgets.IntSlider(min=0, max=dset_length, step=1, value=0, description='Slice:')
    func = partial(adjusted_display_images, dset=dset)
    widget_out = widgets.interactive_output(func, {'slice': w})
    with out:
        display(w)
        display(widget_out)

def on_submit_button_clicked(button):
    opt["io_backend"] = {}
    opt["io_backend"]["type"] = "disk"

    # Clear existing output
    out.clear_output(wait=True)

    # Reset globals (if needed) and print message
    with out:
        print("Dataset generated!")
        pprint.pprint(opt)
    dset = PairedImageDataset(opt)
    # Initialize and display the slider
    initialize_and_display_slider(dset)

# Create the Submit button
submit_button = widgets.Button(description="Submit")
submit_button.on_click(on_submit_button_clicked)

# Add the button to your list of widgets to display
widgets_list.append(submit_button)

# Display all the widgets
opt_widget = widgets.VBox(widgets_list)
display(opt_widget)
# Make sure to display your main output widget in your main layout
display(out)